1. ## Import Libraries

In [129]:
import pandas as pd
import numpy as np
import random
from numpy import inf

2. ## Parameter Setting

In [131]:
#PA in ACO 
init_pheromone = 0
max_iter = 1
ant = 10
rho = 0.05 # Pheromone Evaporation Rate
alpha = 1 # Pheromone Exponential Parameter
beta = 1 # Desirabillity Exponential Parameter
pheromone_scale = 1


3. ## Import Data Set (Supply Chain Logistics Problem)

In [132]:
dataOL = pd.read_excel(r'sclp.xlsx', sheet_name='OrderList')
dataFR = pd.read_excel(r'sclp.xlsx', sheet_name='FreightRates')
dataWhCost = pd.read_excel(r'sclp.xlsx', sheet_name='WhCosts')
dataWhCap = pd.read_excel(r'sclp.xlsx', sheet_name='WhCapacities')


4. ## Data Pre-Processing

In [133]:
#1 OrderList

dataOL=dataOL.drop(columns=['Order Date','TPT','Ship ahead day count','Ship Late Day count'])

#2 Freight Rate
dataFR = dataFR[dataFR['Carrier'].isin(['V444_0','V444_1','V44_3'])]
# print(p_origin_port_availability)
dataFR = dataFR[dataFR['orig_port_cd'].isin(['PORT04','PORT05','PORT09'])]
dataFR = dataFR.drop(columns=['tpt_day_cnt','Carrier type'])
print(dataFR.groupby(['orig_port_cd','Carrier']).size().reset_index(name='Availability'))
#3 Warehouse Cost
dataWhCost = dataWhCost[dataWhCost['WH'].isin(['PLANT03','PLANT04','PLANT08','PLANT09','PLANT12','PLANT13','PLANT16'])]

#4 Warehouse Capacity
dataWhCap = dataWhCap[dataWhCap['Plant ID'].isin(['PLANT03','PLANT04','PLANT08','PLANT09','PLANT12','PLANT13','PLANT16'])]


  orig_port_cd Carrier  Availability
0       PORT04  V444_0            20
1       PORT04  V444_1            40
2       PORT05  V444_1            48
3       PORT09  V444_0             2


5. Main Function

In [183]:
local_ant = 3
max_iter = 1
max_util = 10

#Records
record_global_ant = list()

p_origin_port_availability=[]
p_plant_availability=[]
path_order_availability=[]
carrier_availability=[]


#Colony Iteration
for colony_iter in range(max_iter):
    #Local Ants Iteration 
    for _ in range(local_ant):

        dataFR_sub = dataFR
        dataOL_sub = dataOL
        dataWhCap_sub = dataWhCap
        #Utilization
        carrier_utilization,crf_utilization,total_utilization = 0,0,0
        order_carrier_combination,selected_carrier = [],[]
        record_local_ant=[[],[]]
        fitness = []
        # record_global_ant = []
        #Status Reset
        dataOL_sub['Status'] = True
        dataFR_sub['Status'] = True

        if colony_iter > 0:
            p_origin_port_availability['Status'] = True
            p_plant_availability['Status'] = True
            path_order_availability['Status'] = True
            carrier_availability['Status'] = True


        f_data_ol_status = dataOL_sub['Status'] == True 

        total_local_cost = 0

        while total_utilization < max_util:

            #PORT

            if colony_iter == 0:
                p_origin_port_availability = dataOL_sub.where(f_data_ol_status).groupby(['Origin Port','Status']).size().reset_index(name='Availability')
                p_origin_port_availability['Pheromone'] = init_pheromone #[PHEROMONE] Pheromone Initialisation for PORT
            selected_port = selection(p_origin_port_availability,'[PORT SELECTION]') # [SELECTION] Selection Process for PORT

            #PLANT CODE
            f_origin_port = p_plant_availability["Origin Port"] == selected_port # [FILTER] Selected Origin Port Filter
            if colony_iter == 0:
                p_plant_availability = dataOL_sub.where(f_data_ol_status).groupby(['Origin Port','Plant Code','Status']).size().reset_index(name='Availability')
                p_plant_availability['Pheromone'] = init_pheromone #[PHEROMONE] Pheromone Initialisation for PLANT CODE
            p_plant_availability = p_plant_availability.where(f_origin_port).groupby(['Plant Code','Status']).sum().reset_index()
            selected_plant_code = selection(p_plant_availability,'[PLANT SELECTION]') # [SELECTION] Selection Process for PLANT CODE

            #ORDER ID
            f_plant_from_dataOL = dataOL_sub["Plant Code"] == selected_plant_code # [FILTER] Selected Plant Filter DataOL
            f_port_from_dataOL = dataOL_sub["Origin Port"] == selected_port # [FILTER] Selected Origin Port Filter DataOL
            # path_order_availability['Availability'] = 1
            if colony_iter == 0:
                path_order_availability = dataOL_sub.where(f_plant_from_dataOL & f_port_from_dataOL & f_data_ol_status).groupby(['Order ID','Service Level','Carrier','Status']).size().reset_index(name='Availability')
                path_order_availability['Pheromone'] = init_pheromone # [PHEROMONE] Pheromone Initialisation for ORDER ID
            selected_path_order = selection(path_order_availability,'[ORDER ID SELECTION]')

            #CARRIER
            use_carrier = True #Does the logistics requires carriers?
            order_carrier_status = False #Has the carrier been used?
            carrier_index,carrier_index_index = 0,0

            f_carrier_carrier = dataFR_sub['Carrier'] == selected_path_order['Carrier'][0] # [FILTER]
            f_carrier_order_origin = dataFR_sub['orig_port_cd'] == selected_port# [FILTER]
            if selected_path_order['Carrier'][0] != 'V44_3':
                
                f_carrier_sel = dataFR_sub['Status'] == True
                if colony_iter == 0:
                    carrier_availability = dataFR_sub.where(f_carrier_order_origin & f_carrier_sel).groupby([dataFR_sub.index,'Carrier','orig_port_cd','svc_cd','minm_wgh_qty','max_wgh_qty','Status']).size().reset_index(name='Availability')
                    carrier_availability['Availability'] = 1
                    carrier_availability['Pheromone'] = init_pheromone # Pheromone Initialisation for Carrier
                selected_carrier = selection(carrier_availability,'[CARRIER SELECTION]')

            else:
                use_carrier = False #No Carriers Needed
                selected_carrier = 'CRF'
            temp_occ = []
            order_carrier_status = False
            if use_carrier:
                if order_carrier_combination:
                    for index,occ in enumerate(order_carrier_combination):
                        if order_carrier_status == False:
                            if selected_path_order['Order ID'][0] == occ[0][0] and selected_path_order['Carrier'][0] == occ[0][1]:
                                # print('HERE3')
                                if order_carrier_status == False:
                                    for index_,c_index in enumerate(occ[3]):
                                        if c_index[0] == selected_carrier['level_0'][0]:
                                            order_carrier_status = True
                                            break

                                            # print('HERE2')
                                    if not order_carrier_status:
                                        idx_lst = [selected_carrier['level_0'][0],init_pheromone,'Unoccupied']
                                        temp_occ[index][3].append(idx_lst)
                                        order_carrier_status = True
                                        # print('HERE')
                                        carrier_index_index = index_
                                        break
                                    
                                    else:
                                        if temp_occ[index][3][index_][2] == 'Occupied':
                                            print('[Carrier Is Occupied]')      

                            else:
                                genesis_list = []
                                #print meow
                                idx_lst = [selected_carrier['level_0'][0],init_pheromone,'Unoccupied']
                                genesis_list.append(idx_lst)
                                order_carrier = [selected_path_order['Order ID'][0],selected_path_order['Carrier'][0],selected_carrier['Size'][0],genesis_list]
                                temp_occ.append(order_carrier)
                                carrier_index = index
                                order_carrier_status = True
                                break
                                

                else:
                    genesis_list = []
                    idx_lst = [selected_carrier['level_0'][0],init_pheromone,'Unoccupied']
                    genesis_list.append(idx_lst)
                    order_carrier = [selected_path_order['Order ID'][0],selected_path_order['Carrier'][0],selected_carrier['Size'][0],genesis_list]
                    temp_occ.append(order_carrier)
                    

            #COST CALCULATION
            total_cost = 0
            if(use_carrier):
                t_cost = transportation_cost(use_carrier,selected_path_order,selected_carrier,dataOL_sub,dataFR_sub)
            else:
                t_cost = 0.00

            if(type(t_cost) is not str):
                wh_cost,dataWhCap_sub = warehouse_cost(dataWhCost,dataWhCap_sub,selected_path_order,selected_plant_code,dataOL_sub)
                # print('[TRANSPORTATION COST]\t$',"{:.2f}".format(t_cost))
                if(type(wh_cost) is not str):
                    total_cost = t_cost+wh_cost
                else:
                    total_cost = 'INVALID ROUTE'
                    temp_occ[carrier_index][3][carrier_index_index][2] = 'INVALID'
            else:
                # print('[TRANSPORTATION COST]\t$',t_cost)
                total_cost = 'INVALID ROUTE'
                temp_occ[carrier_index][3][carrier_index_index][2] = 'INVALID'

            # print('[WAREHOUSE COST]\t$',"{:.2f}".format(wh_cost))
                
            # if(type(total_cost) is str):
                # print('[TOTAL COST]\t\t$',total_cost,' due to ',t_cost)
            # else:
                # print('[TOTAL COST]\t\t$',"{:.2f}".format(total_cost))


            #AVAILABILITY UPDATE
            if(type(t_cost) is not str and type(wh_cost) is not str ):
                if (type(selected_carrier) is str and crf_utilization < (max_util-carrier_utilization)):
                    data_OL_index = dataOL[dataOL['Order ID']==selected_path_order['Order ID'][0]].index.values #filter index finding
                    dataOL.at[data_OL_index,'Status'] = False
                    record_local_ant[1].append(selected_path_order['Order ID'][0])
                    crf_utilization+=1
                    total_utilization+=1
                
                if use_carrier:
                    if temp_occ[carrier_index][3][carrier_index_index][2] == 'Unoccupied' and carrier_utilization < 110:
                        #Order ID Status
                        data_OL_index = dataOL[dataOL['Order ID']==selected_path_order['Order ID'][0]].index.values #filter index finding
                        dataOL.at[data_OL_index,'Status'] = False
                        #Carrier Status
                        dataFR_sub.at[selected_carrier['level_0'][0],'Status'] = False
                        # if use_carrier:
                        temp_occ[carrier_index][3][carrier_index_index][2] = 'Occupied' ##carrier tu nanti kena filter by occupied
                        # temp_occ.append(total_cost)
                        order_carrier_combination.append(temp_occ)
                        record_local_ant[0].append(temp_occ)
                        carrier_utilization+=1
                        total_utilization+=1
                
            # print(order_carrier_combination)
                print(total_utilization,' [ ',selected_path_order['Order ID'][0],' ] ', total_cost)
                total_local_cost += total_cost
            # print('=================================================')

        print("TOTAL LOCAL COST: $",total_local_cost)
        temp_loc = [record_local_ant,total_local_cost]
        record_global_ant.append(temp_loc)

    #Fitness/Objective Function
    # record_colony.append(record_global_ant)
    lowest_cost,lowest_cost_index,fitness = objective_function(record_global_ant,rho,dataOL_sub,dataFR_sub,p_origin_port_availability,p_plant_availability,path_order_availability,carrier_availability)
    
    # dataOL_sub,dataFR_sub = pheromone_update(record_global_ant,rho,)

            # p_origin_port_availability
            # p_plant_availability
            # path_order_availability
            # carrier_availability




    #PHEROMONE UPDATE
    #QUEEN ANT


1  [  1447326023.7  ]  743.5269854132529
2  [  1447390850.7  ]  149.11914671157965
3  [  1447398478.7  ]  1867.9727051404177
4  [  1447274595.7  ]  301.8840726067039
5  [  1447157930.7  ]  2724.480146166678
6  [  1447186964.7  ]  367.49930943735507
7  [  1447144040.7  ]  131.61247668961636
8  [  1447305209.7  ]  1028.2762586597926
9  [  1447373487.7  ]  125.75295966498722
10  [  1447305654.7  ]  389.00583060198886
TOTAL LOCAL COST: $ 7829.129891092373
1  [  1447390850.7  ]  149.11914671157965
2  [  1447158018.7  ]  819.7578058529891
3  [  1447320236.7  ]  4286.930165034484
4  [  1447213716.7  ]  756.9656379682199
5  [  1447219266.7  ]  7514.126585734424
6  [  1447326023.7  ]  743.5269854132529
7  [  1447377315.7  ]  2263.4530517580192
8  [  1447155058.7  ]  5765.181946080857
9  [  1447257230.7  ]  1067.412974365953
10  [  1447308589.7  ]  2695.4097410248164
TOTAL LOCAL COST: $ 26061.8840399446
1  [  1447390850.7  ]  149.11914671157965
2  [  1447191277.7  ]  5730.625410939514
3  [  1447

In [247]:


# dat['pheromone'] = (fitness[x]+data['pheromone'])*(1-rho)
# f_port = p_origin_port_availability['Status'] = True
# print(p_origin_port_availability)
# # port_aggregate = {'Origin Port':'sum'}
# # df_new = df.groupby
# print(p_origin_port_availability.where(f_port))

for index_rga,rga in enumerate(record_global_ant):
        # crf_len = len(rga[index][0][0][1])
        carrier_len = len(rga[index][0][0][0])
        print(rga[0][1])
        print()

        ol_dat = dataOL[dataOL['Order ID'].isin(rga[0][1])] #CRF
        ol_dat = ol_dat.groupby('Origin Port').size()
        print(ol_dat[''])




[1447390850.7, 1447398478.7, 1447305209.7, 1447373487.7, 1447305654.7]



KeyError: ''

In [210]:
# print(wh_cost)
# print(selected_path_order)
# print(order_carrier_combination)
print(record_global_ant[0][0][1])
# #CRF
print()
#id
print((record_global_ant[0][0][0][0][0][0]))
#pheromone
print()
print((record_global_ant[0][0][0][0][0][3][0][1]))
# print()
# #Carrier Sekali -Order ID
# print(record_global_ant[1][0][0][0][0][3][0][0])
# print(len(record_global_ant[1][0][0]))
# print()
# print(record_global_ant[0])
# print(record_global_ant[1])
# print()
# print(record_global_ant)
# dataOL[dataOL['Order ID'].isin(record_global_ant[0][0][1])]
# print('mahu combine')
# combined = list(set(record_global_ant[0]+record_global_ant[1]))
# print()
# print(record_global_ant[0])
# print()
# print(combined)

[1447390850.7, 1447398478.7, 1447305209.7, 1447373487.7, 1447305654.7]

1447326023.7

0


6. ## Functions

In [6]:
def selection(dat,type_sel):
    # print('============================================================================================================================')
    # print(dat)
    #parameter sequence: target, availability, pheromone
    
    #pheromone
    cumulative_pheromone,cum_p_segment = 0,0
    start_range,end_range = 0,0
    selection_list = []
    data_length = 0
    probability = 0
    target,target_index = '',0

    # Eliminate Routes With False Status

    f_status_count = dat['Status'] == True
    dat = dat.where(f_status_count)
    dat = dat.dropna()
    data_length = len(dat.index)
    cumulative_pheromone = dat['Pheromone'].sum()

    
    #Normalize the Routes in a Uniform Roulette Wheel
    for index,row in dat.iterrows():

        cur_list=[]
        # Will be executed during the first iteration of ACO
        if dat['Pheromone'].sum() == 0:
            end_range = (index+1)/data_length
        # The next iterations
        else:
            # .index
            end_range = (cum_p_segment+row['Pheromone']/cumulative_pheromone)
            cum_p_segment += row['Pheromone']
            
        cur_list = [row[0],start_range,end_range]
        selection_list.append(cur_list)
        start_range = end_range

    roulette_value = random.uniform(0, 1)
    for sel_ in selection_list:
        if sel_[1] <= roulette_value and sel_[2] >= roulette_value:
            probability = "{:.2f}".format((sel_[2]-sel_[1])*100)
            target = sel_[0]

    # print(type_sel,'\tSelected Target is ',target,' with the Probability of ',probability,'% [ Roulette Value : ',roulette_value," ]")
    if dat.columns[0] == 'Order ID':
        f_order = dat['Order ID'] == target
        target = dat.where(f_order).groupby(['Order ID','Service Level','Carrier']).sum().reset_index()
        target['Size'] = data_length
        
    elif dat.columns[1] == 'Carrier':
        f_carrier = dat['level_0'] == target
        target = dat.where(f_carrier).groupby(['level_0','Carrier']).sum().reset_index()
        target['Size'] = data_length
        
    # print(target)

    return target

In [144]:
def pheromone_update(record_global_ant,rho,dataOL_sub,dataFR_sub,p_origin_port_availability,p_plant_availability,path_order_availability,carrier_availability):


    
    for index_rga,rga in enumerate(record_global_ant):
        # crf_len = len(rga[index][0][0][1])
        carrier_len = len(rga[index][0][0][0])


        dataOL[dataOL['Order ID'].isin(record_global_ant[0][0][1])]

    
    pheromone = 0
    return dataOL_sub,dataFR_sub

In [115]:
def objective_function(record_global_ant):

    lowest_cost,highest_cost = record_global_ant[0][-1],record_global_ant[0][-1]
    lowest_cost_index,highest_cost_index = 0,0
    fitness = []

    for index_rga,rga in enumerate(record_global_ant):
        if rga[-1] < lowest_cost:
            lowest_cost = rga[-1]
            lowest_cost_index = index_rga
        
        if rga[-1] > highest_cost:
            highest_cost = rga[-1]
            highest_cost_index = index_rga

    for index_rga,rga in enumerate(record_global_ant):
        # fitness_value = 1-(rga[-1]/highest_cost)
        fitness_value = 1/(rga[-1]/lowest_cost)
        fitness.append(fitness_value)

    print('[ ',lowest_cost_index,' ] - LOWEST COST: $',lowest_cost)
    print('\nFitness')
    print(fitness)
    
    return lowest_cost,lowest_cost_index, fitness

In [8]:
def warehouse_cost(dataWhCost,dataWhCap,selected_path_order,selected_plant_code,dataOL):

    data_Cap = dataWhCap.loc[dataWhCap['Plant ID'] == selected_plant_code].reset_index()
    data_Cap_index = dataWhCap.loc[dataWhCap['Plant ID'] == selected_plant_code]
    data_Cap_index = data_Cap_index.index[0]
    data_Cost = dataWhCost.loc[dataWhCost['WH'] == selected_plant_code].reset_index()
    data_OL = dataOL.loc[dataOL['Order ID'] == selected_path_order['Order ID'][0]].reset_index()
    wh_cost = 0

    if data_Cap['Daily Capacity '][0] > 0:
        counter = data_Cap['Daily Capacity '][0]-1
        wh_cost = data_OL['Unit quantity'][0]*data_Cost['Cost/unit'][0]
        dataWhCap.at[data_Cap_index,'Daily Capacity '] = counter

    else:
        wh_cost = 'Warehouse is Full!'

    return wh_cost,dataWhCap

In [9]:
def transportation_cost(use_carrier,selected_path_order,selected_carrier,dataOL,dataFR_sub):
    
    transportation_cost = 0
    data_OL = dataOL.loc[dataOL['Order ID'] == selected_path_order['Order ID'][0]].reset_index()
    data_FR = dataFR_sub.loc[selected_carrier['level_0'][0]]

    if use_carrier:

        if data_OL['Weight'][0] < data_FR['minm_wgh_qty']:
            transportation_cost = 'Not Enough Weight'
        elif data_OL['Weight'][0] > data_FR['max_wgh_qty']:
            transportation_cost = 'Exceeded Weight'
            
        else:
            if data_FR['mode_dsc'] == 'GROUND':
                transportation_cost = (data_FR['rate']/data_FR['max_wgh_qty'])*data_OL['Weight'][0]
            else:
                transportation_cost = data_FR['rate']*data_OL['Weight'][0]

            if transportation_cost < data_FR['minimum cost']:
                transportation_cost = data_FR['minimum cost']

    return transportation_cost